In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Concatenate, concatenate, Dropout, Reshape, dot, Dot
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, mean_squared_error
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

Using TensorFlow backend.
d:\anaconda3\envs\soojin\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\soojin\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\soojin\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\soojin\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
X = np.load('./datas/X_data.npy')
y = np.load('./datas/y_data.npy') 

In [7]:
def keras_model():
    user_vector_input = Input(shape=(128, ))
    
    dense_u_v = Dense(128, activation = 'relu')(user_vector_input)
    dense_u_v = Dropout(0.5)(dense_u_v)
    dense_u_v = Dense(64, activation = 'relu')(dense_u_v)
    dense_u_v = Dropout(0.5)(dense_u_v)
    dense_u_v = Dense(32, activation = 'relu')(dense_u_v)
    dense_u_v = Dropout(0.5)(dense_u_v)
    dense_u_v = Dense(16, activation = 'relu')(dense_u_v)
    dense_u_v = Dense(1, activation = 'sigmoid')(dense_u_v)
    
    model = Model(inputs=user_vector_input, outputs=dense_u_v)
    model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics=['acc'])
    return model


In [8]:
model = keras_model()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state=1)

In [14]:
modelpath = './datas/recommender.model'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [15]:
model.fit(X_train, y_train, validation_split = 0.2, epochs=50, batch_size=64, callbacks=[early_stop, checkpointer])

Train on 72457 samples, validate on 18115 samples
Epoch 1/50
72457/72457 [==============================] - 15s 207us/step - loss: 0.6417 - acc: 0.6030 - val_loss: 0.6135 - val_acc: 0.6306

Epoch 00001: val_loss improved from inf to 0.61350, saving model to ./datas/recommender.model
Epoch 2/50
72457/72457 [==============================] - 7s 94us/step - loss: 0.6071 - acc: 0.6261 - val_loss: 0.5916 - val_acc: 0.6601

Epoch 00002: val_loss improved from 0.61350 to 0.59162, saving model to ./datas/recommender.model
Epoch 3/50
72457/72457 [==============================] - 7s 102us/step - loss: 0.5899 - acc: 0.6461 - val_loss: 0.5665 - val_acc: 0.6895

Epoch 00003: val_loss improved from 0.59162 to 0.56651, saving model to ./datas/recommender.model
Epoch 4/50
72457/72457 [==============================] - 8s 104us/step - loss: 0.5710 - acc: 0.6716 - val_loss: 0.5612 - val_acc: 0.7101

Epoch 00004: val_loss improved from 0.56651 to 0.56123, saving model to ./datas/recommender.model
Epoch 

In [16]:
pred = model.predict(X_test)

In [17]:
pred_label = [1 if i > 0.5 else 0 for i in pred]

In [18]:
print(f1_score(y_test, pred_label))
print(precision_score(y_test, pred_label))
print(recall_score(y_test, pred_label))
print(accuracy_score(y_test, pred_label))

0.7108376715570278
0.5794753086419753
0.9192166462668299
0.7031753742878594
